In [1]:
import numpy as np
import pickle
import os

seed = 2023

In [2]:
import torch

# set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [3]:
from nlpsig_networks.scripts.seqsignet_attention_encoder_functions import (
    seqsignet_attention_encoder_hyperparameter_search,
)

In [4]:
output_dir = "rumours_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## Rumours

In [5]:
%run load_sbert-embeddings.py

In [6]:
df_rumours.head()

,id,label,datetime,text,timeline_id,set
0,5.249902e+17,0,2014-10-22 18:26:23,Police have clarified that there were two shoo...,0,train
1,5.249906e+17,0,2014-10-22 18:27:58,"@CTVNews you guys ""confirmed"" there were 3 sho...",0,train
2,5.249908e+17,1,2014-10-22 18:28:46,@CTVNews get it right. http://t.co/GHYxMuzPG9,0,train
3,5.249927e+17,1,2014-10-22 18:36:29,RT @CTVNews Police have clarified that there w...,0,train
4,5.250038e+17,1,2014-10-22 19:20:41,@CTVNews @ctvsaskatoon so what happened at Rid...,0,train


# SeqSigNet with Attention Network

In [7]:
features = ["time_encoding", "timeline_index"]
standardise_method = ["z_score", None]
include_features_in_path = True
include_features_in_input = True

In [8]:
split_ids = torch.tensor(df_rumours['timeline_id'].astype(int))

In [9]:
num_epochs = 100
dimensions = [15]
# define swmhau parameters: (output_channels, sig_depth, num_heads)
swmhau_parameters = [(12, 3, 10), (8, 4, 6)]
num_layers = [1]
ffn_hidden_dim_sizes = [[256,256],[512,512]]
dropout_rates = [0.1, 0.2]
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [1, 12, 123]
loss = "focal"
gamma = 2
validation_metric = "f1"
patience = 5

In [10]:
kwargs = {
    "num_epochs": num_epochs,
    "df": df_rumours,
    "id_column": "timeline_id",
    "label_column": "label",
    "embeddings": sbert_embeddings,
    "y_data": y_data,
    "output_dim": output_dim,
    "dimensions": dimensions,
    "log_signature": True,
    "pooling": "signature",
    "transformer_encoder_layers": 2,
    "swmhau_parameters": swmhau_parameters,
    "num_layers": num_layers,
    "ffn_hidden_dim_sizes": ffn_hidden_dim_sizes,
    "dropout_rates": dropout_rates,
    "learning_rates": learning_rates,
    "seeds": seeds,
    "loss": loss,
    "gamma": gamma,
    "device": device,
    "features": features,
    "standardise_method": standardise_method,
    "include_features_in_path": include_features_in_path,
    "include_features_in_input": include_features_in_input,
    "split_ids": split_ids,
    "k_fold": True,
    "patience": patience,
    "validation_metric": validation_metric,
    "verbose": False,
}

# history_length=11

In [11]:
shift = 3
window_size = 5
n = 3

## Random Projections

In [12]:
(
    seqsignet_attention_encoder_umap_11,
    best_seqsignet_attention_encoder_umap_11,
    _,
    __,
) = seqsignet_attention_encoder_hyperparameter_search(
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/seqsignet_attention_encoder_umap_focal_{gamma}_{shift}_{window_size}_{n}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
given shift 3, window size 5 and n 3: history length = 11
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

given shift 3, window size 5 and n 3: history length = 11
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in rumours_output/seqsignet_attention_encoder_umap_focal_2_3_5_3_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in rumours_output/seqsignet_attention_encoder_umap_focal_2_3_5_3_kfold_best_model.csv


In [13]:
seqsignet_attention_encoder_umap_11.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

/tmp/ipykernel_1358432/2250631359.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  seqsignet_attention_encoder_umap_11.groupby(


accuracy  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         8               4         6         1          (256, 256)     0.1          0.0001         0.685617   
                                                                                      0.0005         0.697794   
                                                                                      0.0010         0.701522   
                                                                         0.2          0.0001         0.696738   
                                                                                      0.0005         0.697919   
                                                                                      0.0010         0.700093   
                                                          (512, 512)     0.1          0.0001         0.696117   
                                                                                      0.0005         0.697049   
                                                                                      0.0010         0.694812   
                                                                         0.2          0.0001         0.697111   
                                                                                      0.0005         0.701833   
                                                                                      0.0010         0.694874   
           12              3         10        1          (256, 256)     0.1          0.0001         0.681267   
                                                                                      0.0005         0.704629   
                                                                                      0.0010         0.698913   
                                                                         0.2          0.0001         0.688661   
                                                                                      0.0005         0.695620   
                                                                                      0.0010         0.705747   
                                                          (512, 512)     0.1          0.0001         0.674060   
                                                                                      0.0005         0.696552   
                                                                                      0.0010         0.699658   
                                                                         0.2          0.0001         0.692637   
                                                                                      0.0005         0.694812   
                                                                                      0.0010         0.699348   

                                                                                                           f1  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim dropout_rate learning_rate             
15         8               4         6         1          (256, 256)     0.1          0.0001         0.655365   
                                                                                      0.0005         0.668842   
                                                                                      0.0010         0.670383   
                                                                         0.2          0.0001         0.645966   
                                                                                      0.0005         0.667709   
                                                                                      0.0010         0.669096   
                                                          (512, 512)     0.1          0.0001         0.658541   
                                                                                      0.0005         0.669398   
                                

In [14]:
best_seqsignet_attention_encoder_umap_11

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,learning_rate,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.687232,0.665652,"[0.7505945303210464, 0.5807096451774113]",0.663072,"[0.779320987654321, 0.5468235294117647]",0.671492,"[0.7239105504587156, 0.619072988811934]",None,0.760615,...,0.0001,1,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.683877,0.666668,"[0.7424058323207777, 0.5909310178485286]",0.664646,"[0.789405684754522, 0.5398854120758043]",0.676663,"[0.7006880733944955, 0.6526371870005327]",None,0.766411,...,0.0001,12,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.672693,0.655122,"[0.7329683698296836, 0.5772749157438614]",0.653505,"[0.780440414507772, 0.5265700483091788]",0.664863,"[0.6909403669724771, 0.6387852956846031]",None,0.768432,...,0.0001,123,focal,2,True,5,Conv1d,None,concatenation,64


In [15]:
best_seqsignet_attention_encoder_umap_11[
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
]

,dimensions,output_channels,sig_depth,num_heads,num_layers,ffn_hidden_dim,dropout_rate,learning_rate
0,15,12,3,10,1,"(256, 256)",0.1,0.0001
0,15,12,3,10,1,"(256, 256)",0.1,0.0001
0,15,12,3,10,1,"(256, 256)",0.1,0.0001


in path: 0.658

in input: 0.6430702845314854

both: 0.6596275491973381

In [16]:
best_seqsignet_attention_encoder_umap_11["f1"].mean()

0.6624807185402183

In [17]:
best_seqsignet_attention_encoder_umap_11["precision"].mean()

0.6604076794522271

In [18]:
best_seqsignet_attention_encoder_umap_11["recall"].mean()

0.6710057437204596

In [19]:
np.stack(best_seqsignet_attention_encoder_umap_11["f1_scores"]).mean(axis=0)

array([0.74198958, 0.58297186])

In [20]:
np.stack(best_seqsignet_attention_encoder_umap_11["precision_scores"]).mean(axis=0)

array([0.7830557 , 0.53775966])

In [21]:
np.stack(best_seqsignet_attention_encoder_umap_11["recall_scores"]).mean(axis=0)

array([0.70517966, 0.63683182])

# history_length=20

In [22]:
shift = 3
window_size = 5
n = 6

## Random Projections

In [ ]:
(
    seqsignet_attention_encoder_umap_20,
    best_seqsignet_attention_encoder_umap_20,
    _,
    __,
) = seqsignet_attention_encoder_hyperparameter_search(
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/seqsignet_attention_encoder_umap_focal_{gamma}_{shift}_{window_size}_{n}_kfold.csv",
    **kwargs,
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
given shift 3, window size 5 and n 6: history length = 20
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/5568 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


...

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
seqsignet_attention_encoder_umap_20.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

In [ ]:
best_seqsignet_attention_encoder_umap_20

In [ ]:
best_seqsignet_attention_encoder_umap_20[
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
]

In [ ]:
best_seqsignet_attention_encoder_umap_20["f1"].mean()

In [ ]:
best_seqsignet_attention_encoder_umap_20["precision"].mean()

In [ ]:
best_seqsignet_attention_encoder_umap_20["recall"].mean()

In [ ]:
np.stack(best_seqsignet_attention_encoder_umap_20["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_seqsignet_attention_encoder_umap_20["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_seqsignet_attention_encoder_umap_20["recall_scores"]).mean(axis=0)

# history_length=35

In [ ]:
shift = 3
window_size = 5
n = 11

## Random Projections

In [ ]:
(
    seqsignet_attention_encoder_umap_35,
    best_seqsignet_attention_encoder_umap_35,
    _,
    __,
) = seqsignet_attention_encoder_hyperparameter_search(
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/seqsignet_attention_encoder_umap_focal_{gamma}_{shift}_{window_size}_{n}_kfold.csv",
    **kwargs,
)

In [ ]:
seqsignet_attention_encoder_umap_35.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

In [ ]:
best_seqsignet_attention_encoder_umap_35

In [ ]:
best_seqsignet_attention_encoder_umap_35[
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
]

In [ ]:
best_seqsignet_attention_encoder_umap_35["f1"].mean()

In [ ]:
best_seqsignet_attention_encoder_umap_35["precision"].mean()

In [ ]:
best_seqsignet_attention_encoder_umap_35["recall"].mean()

In [ ]:
np.stack(best_seqsignet_attention_encoder_umap_35["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_seqsignet_attention_encoder_umap_35["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_seqsignet_attention_encoder_umap_35["recall_scores"]).mean(axis=0)

# history_length=80

In [ ]:
shift = 3
window_size = 5
n = 26

## UMAP

## Random Projections

In [ ]:
(
    seqsignet_attention_encoder_umap_80,
    best_seqsignet_attention_encoder_umap_80,
    _,
    __,
) = seqsignet_attention_encoder_hyperparameter_search(
    shift=shift,
    window_size=window_size,
    n=n,
    dim_reduce_methods=["umap"],
    results_output=f"{output_dir}/seqsignet_attention_encoder_umap_focal_{gamma}_{shift}_{window_size}_{n}_kfold.csv",
    **kwargs,
)

In [ ]:
seqsignet_attention_encoder_umap_80.groupby(
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
).mean()

In [ ]:
best_seqsignet_attention_encoder_umap_80

In [ ]:
best_seqsignet_attention_encoder_umap_80[
    [
        "dimensions",
        "output_channels",
        "sig_depth",
        "num_heads",
        "num_layers",
        "ffn_hidden_dim",
        "dropout_rate",
        "learning_rate",
    ]
]

In [ ]:
best_seqsignet_attention_encoder_umap_80["f1"].mean()

In [ ]:
best_seqsignet_attention_encoder_umap_80["precision"].mean()

In [ ]:
best_seqsignet_attention_encoder_umap_80["recall"].mean()

In [ ]:
np.stack(best_seqsignet_attention_encoder_umap_80["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_seqsignet_attention_encoder_umap_80["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_seqsignet_attention_encoder_umap_80["recall_scores"]).mean(axis=0)